In [1]:
import os
from random import shuffle
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import datetime
from datetime import timedelta
import psycopg2
import tidalUtl.InterfaceSQL as isq
import tidalUtl.PrpUtl as prp

In [2]:
#config
DBNAME = "Splatoon"

# Create ExecData

In [4]:
isq.createExecData(DBNAME)

DONE.


In [6]:
#すでに作られている場合はエラーが返る。
isq.createExecData(DBNAME)

DuplicateTable: リレーション"execdata_train"はすでに存在します


# Delete ExecData

In [3]:
isq.DeleteExecData(DBNAME)

DONE.


# Read ExecData

In [5]:
#全table
df = isq.selectDataTable(DBNAME, "execdata_train", clmns = None)

In [6]:
df

,id,lobby_mode,mode,stage,a1_weapon,a1_rank,a1_level,a2_weapon,a2_rank,a2_level,...,b3_rank,b3_level,b4_weapon,b4_rank,b4_level,yyyymmdd,time,datedelta,hour,y
0,1,regular,nawabari,sumeshi,sshooter_becchu,None,139,soytuber_custom,None,118,...,None,68,sharp_neo,None,31,2019-10-15,20:00:00,78,20,1
1,2,regular,nawabari,arowana,parashelter_sorella,None,198,jetsweeper,None,77,...,None,168,sputtery_clear,None,151,2019-12-14,04:00:00,18,4,0
2,3,gachi,hoko,ama,nzap89,a-,114,quadhopper_black,a,68,...,a-,160,dualsweeper_custom,a,126,2019-12-25,14:00:00,7,14,0
3,4,regular,nawabari,engawa,bamboo14mk1,None,336,splatroller_becchu,None,131,...,None,194,hotblaster_custom,None,391,2019-11-11,14:00:00,51,14,0
4,5,gachi,hoko,chozame,bold_7,x,299,hissen_hue,x,97,...,x,246,wakaba,x,160,2019-12-14,06:00:00,18,6,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
66120,66121,gachi,area,devon,hokusai_becchu,a,66,soytuber,a,113,...,a,38,wakaba,a+,283,2019-10-11,10:00:00,82,10,1
66121,66122,gachi,asari,otoro,hokusai_becchu,s+,204,prime_becchu,s+,96,...,s+,188,nzap89,s+,170,2019-12-04,00:00:00,28,0,1
66122,66123,gachi,hoko,bbass,prime_collabo,a+,116,promodeler_rg,a,86,...,a+,191,hissen_hue,a+,139,2019-10-22,00:00:00,71,0,1
66123,66124,gachi,area,fujitsubo,heroroller_replica,s+,210,sharp_neo,s+,96,...,s+,223,sharp_neo,s+,194,2019-12-11,00:00:00,21,0,0


In [8]:
#一部列以外を検索。
#まず列をロード。
col = isq.readColumns(DBNAME, "execdata_train")
#col指定
col = col[col.cols != "id"].reset_index()
df = isq.selectDataTable(DBNAME, "execdata_train", clmns = col)

In [9]:
df

,lobby_mode,mode,stage,a1_weapon,a1_rank,a1_level,a2_weapon,a2_rank,a2_level,a3_weapon,...,b3_rank,b3_level,b4_weapon,b4_rank,b4_level,yyyymmdd,time,datedelta,hour,y
0,regular,nawabari,sumeshi,sshooter_becchu,None,139,soytuber_custom,None,118,pablo_hue,...,None,68,sharp_neo,None,31,2019-10-15,20:00:00,78,20,1
1,regular,nawabari,arowana,parashelter_sorella,None,198,jetsweeper,None,77,campingshelter_camo,...,None,168,sputtery_clear,None,151,2019-12-14,04:00:00,18,4,0
2,gachi,hoko,ama,nzap89,a-,114,quadhopper_black,a,68,prime_becchu,...,a-,160,dualsweeper_custom,a,126,2019-12-25,14:00:00,7,14,0
3,regular,nawabari,engawa,bamboo14mk1,None,336,splatroller_becchu,None,131,dynamo_tesla,...,None,194,hotblaster_custom,None,391,2019-11-11,14:00:00,51,14,0
4,gachi,hoko,chozame,bold_7,x,299,hissen_hue,x,97,h3reelgun_d,...,x,246,wakaba,x,160,2019-12-14,06:00:00,18,6,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
66120,gachi,area,devon,hokusai_becchu,a,66,soytuber,a,113,wakaba,...,a,38,wakaba,a+,283,2019-10-11,10:00:00,82,10,1
66121,gachi,asari,otoro,hokusai_becchu,s+,204,prime_becchu,s+,96,prime_becchu,...,s+,188,nzap89,s+,170,2019-12-04,00:00:00,28,0,1
66122,gachi,hoko,bbass,prime_collabo,a+,116,promodeler_rg,a,86,hissen,...,a+,191,hissen_hue,a+,139,2019-10-22,00:00:00,71,0,1
66123,gachi,area,fujitsubo,heroroller_replica,s+,210,sharp_neo,s+,96,dualsweeper_custom,...,s+,223,sharp_neo,s+,194,2019-12-11,00:00:00,21,0,0


In [10]:
#正規表現を用いた検索。
col = isq.readColumns(DBNAME, "execdata_train")
#検索機能を使って当てはまるものを抽出。
col = col[(col["cols"].str.contains('a[0-9]_')) | (col["cols"].str.contains('b[0-9]_'))].reset_index()
df = isq.selectDataTable(DBNAME, "execdata_train", clmns = col)

In [11]:
df

,a1_weapon,a1_rank,a1_level,a2_weapon,a2_rank,a2_level,a3_weapon,a3_rank,a3_level,a4_weapon,...,b1_level,b2_weapon,b2_rank,b2_level,b3_weapon,b3_rank,b3_level,b4_weapon,b4_rank,b4_level
0,sshooter_becchu,None,139,soytuber_custom,None,118,pablo_hue,None,13,hokusai,...,28,hokusai_becchu,None,26,herocharger_replica,None,68,sharp_neo,None,31
1,parashelter_sorella,None,198,jetsweeper,None,77,campingshelter_camo,None,198,nzap85,...,83,squiclean_b,None,118,campingshelter,None,168,sputtery_clear,None,151
2,nzap89,a-,114,quadhopper_black,a,68,prime_becchu,a,225,jetsweeper,...,50,nzap85,a+,163,prime_becchu,a-,160,dualsweeper_custom,a,126
3,bamboo14mk1,None,336,splatroller_becchu,None,131,dynamo_tesla,None,189,prime_becchu,...,273,liter4k,None,189,promodeler_mg,None,194,hotblaster_custom,None,391
4,bold_7,x,299,hissen_hue,x,97,h3reelgun_d,x,96,splatroller,...,101,sputtery_hue,x,45,bucketslosher_soda,x,246,wakaba,x,160
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
66120,hokusai_becchu,a,66,soytuber,a,113,wakaba,a+,114,nzap83,...,509,nzap85,a+,272,splatcharger,a,38,wakaba,a+,283
66121,hokusai_becchu,s+,204,prime_becchu,s+,96,prime_becchu,s+,112,momiji,...,163,prime_becchu,s+,167,pablo,s+,188,nzap89,s+,170
66122,prime_collabo,a+,116,promodeler_rg,a,86,hissen,a-,19,sshooter_becchu,...,205,bold_neo,a+,70,nzap85,a+,191,hissen_hue,a+,139
66123,heroroller_replica,s+,210,sharp_neo,s+,96,dualsweeper_custom,s+,126,prime_becchu,...,91,heroroller_replica,s+,395,rapid_elite_deco,s+,223,sharp_neo,s+,194


# Preprocessing: Update, AddColumn

In [3]:
#欠損値補完
#train
#まず列名をロード。
col = isq.readColumns(DBNAME, "execdata_train")
#欠損値のある列をロード
col = col[(col.cols == "a1_rank")   | (col.cols == "a2_rank") 
        | (col.cols == "a3_rank")   | (col.cols == "a4_rank") 
        | (col.cols == "a4_weapon") | (col.cols == "a4_level") 
        | (col.cols == "b1_rank")   | (col.cols == "b2_rank") 
        | (col.cols == "b3_rank")   | (col.cols == "b4_rank") 
        | (col.cols == "b3_weapon") | (col.cols == "b3_level") 
        | (col.cols == "b4_weapon") | (col.cols == "b4_level")].reset_index()
train = isq.selectDataTable(DBNAME, "execdata_train", clmns = col)

#int値の欠損値：平均値
mean_A4level = int(train['a4_level'].mean())
mean_B3level = int(train['b3_level'].mean())
mean_B4level = int(train['b4_level'].mean())
#欠損値を補完
train, dfIsNan = prp.FillnaAndInsertIsnan(train, 
                             [('a1_rank','na'), ('a2_rank','na'), ('a3_rank','na'), ('a4_rank','na'), ('a4_weapon','na'), 
                              ('a4_level',mean_A4level), ('b1_rank','na'), ('b2_rank','na'), ('b3_rank','na'), ('b4_rank','na'),
                              ('b3_weapon','na'), ('b3_level',mean_B3level), ('b4_weapon','na'), ('b4_level',mean_B4level)])


In [4]:
train.head(6)

,a1_rank,a2_rank,a3_rank,a4_weapon,a4_rank,a4_level,b1_rank,b2_rank,b3_weapon,b3_rank,b3_level,b4_weapon,b4_rank,b4_level
0,na,na,na,hokusai,na,10,na,na,herocharger_replica,na,68,sharp_neo,na,31
1,na,na,na,nzap85,na,123,na,na,campingshelter,na,168,sputtery_clear,na,151
2,a-,a,a,jetsweeper,a,107,a-,a+,prime_becchu,a-,160,dualsweeper_custom,a,126
3,na,na,na,prime_becchu,na,41,na,na,promodeler_mg,na,194,hotblaster_custom,na,391
4,x,x,x,splatroller,x,136,x,x,bucketslosher_soda,x,246,wakaba,x,160
5,s,s,s,splatroller,s,51,s,s,kelvin525_deco,s,35,nova,s,175


In [5]:
dfIsNan

,a1_rank_isnan,a2_rank_isnan,a3_rank_isnan,a4_rank_isnan,a4_weapon_isnan,a4_level_isnan,b1_rank_isnan,b2_rank_isnan,b3_rank_isnan,b4_rank_isnan,b3_weapon_isnan,b3_level_isnan,b4_weapon_isnan,b4_level_isnan
0,True,True,True,True,False,False,True,True,True,True,False,False,False,False
1,True,True,True,True,False,False,True,True,True,True,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,False,False,False,False
3,True,True,True,True,False,False,True,True,True,True,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
66120,False,False,False,False,False,False,False,False,False,False,False,False,False,False
66121,False,False,False,False,False,False,False,False,False,False,False,False,False,False
66122,False,False,False,False,False,False,False,False,False,False,False,False,False,False
66123,False,False,False,False,False,False,False,False,False,False,False,False,False,False


In [6]:
train.isnull().sum()

a1_rank      0
a2_rank      0
a3_rank      0
a4_weapon    0
a4_rank      0
a4_level     0
b1_rank      0
b2_rank      0
b3_weapon    0
b3_rank      0
b3_level     0
b4_weapon    0
b4_rank      0
b4_level     0
dtype: int64

In [7]:
#ID(主キー)をロード。
#まず列をロード。
col = isq.readColumns(DBNAME, "execdata_train")
#col指定
col = col[col.cols == "id"].reset_index()
pkeyDf = isq.selectDataTable(DBNAME, "execdata_train", clmns = col)

## Update

In [23]:
isq.updateFeatures(DBNAME, "execdata_train", train, pkeyDf)

100%|██████████| 66125/66125 [05:21<00:00, 205.69it/s]

Done.


In [35]:
#全table
df = isq.selectDataTable(DBNAME, "execdata_train", clmns = None)
df

,id,lobby_mode,mode,stage,a1_weapon,a1_rank,a1_level,a2_weapon,a2_rank,a2_level,...,b3_rank,b3_level,b4_weapon,b4_rank,b4_level,yyyymmdd,time,datedelta,hour,y
0,1,regular,nawabari,sumeshi,sshooter_becchu,None,139,soytuber_custom,None,118,...,None,68,sharp_neo,None,31,2019-10-15,20:00:00,78,20,1
1,2,regular,nawabari,arowana,parashelter_sorella,None,198,jetsweeper,None,77,...,None,168,sputtery_clear,None,151,2019-12-14,04:00:00,18,4,0
2,3,gachi,hoko,ama,nzap89,a-,114,quadhopper_black,a,68,...,a-,160,dualsweeper_custom,a,126,2019-12-25,14:00:00,7,14,0
3,4,regular,nawabari,engawa,bamboo14mk1,None,336,splatroller_becchu,None,131,...,None,194,hotblaster_custom,None,391,2019-11-11,14:00:00,51,14,0
4,5,gachi,hoko,chozame,bold_7,x,299,hissen_hue,x,97,...,x,246,wakaba,x,160,2019-12-14,06:00:00,18,6,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
66120,66121,gachi,area,devon,hokusai_becchu,a,66,soytuber,a,113,...,a,38,wakaba,a+,283,2019-10-11,10:00:00,82,10,1
66121,66122,gachi,asari,otoro,hokusai_becchu,s+,204,prime_becchu,s+,96,...,s+,188,nzap89,s+,170,2019-12-04,00:00:00,28,0,1
66122,66123,gachi,hoko,bbass,prime_collabo,a+,116,promodeler_rg,a,86,...,a+,191,hissen_hue,a+,139,2019-10-22,00:00:00,71,0,1
66123,66124,gachi,area,fujitsubo,heroroller_replica,s+,210,sharp_neo,s+,96,...,s+,223,sharp_neo,s+,194,2019-12-11,00:00:00,21,0,0


In [29]:
df.isnull().sum()

id            0
lobby_mode    0
mode          0
stage         0
a1_weapon     0
a1_rank       0
a1_level      0
a2_weapon     0
a2_rank       0
a2_level      0
a3_weapon     0
a3_rank       0
a3_level      0
a4_weapon     0
a4_rank       0
a4_level      0
b1_weapon     0
b1_rank       0
b1_level      0
b2_weapon     0
b2_rank       0
b2_level      0
b3_weapon     0
b3_rank       0
b3_level      0
b4_weapon     0
b4_rank       0
b4_level      0
yyyymmdd      0
time          0
datedelta     0
hour          0
y             0
dtype: int64

## AddColumn

In [8]:
dfIsNan

,a1_rank_isnan,a2_rank_isnan,a3_rank_isnan,a4_rank_isnan,a4_weapon_isnan,a4_level_isnan,b1_rank_isnan,b2_rank_isnan,b3_rank_isnan,b4_rank_isnan,b3_weapon_isnan,b3_level_isnan,b4_weapon_isnan,b4_level_isnan
0,True,True,True,True,False,False,True,True,True,True,False,False,False,False
1,True,True,True,True,False,False,True,True,True,True,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,False,False,False,False
3,True,True,True,True,False,False,True,True,True,True,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
66120,False,False,False,False,False,False,False,False,False,False,False,False,False,False
66121,False,False,False,False,False,False,False,False,False,False,False,False,False,False
66122,False,False,False,False,False,False,False,False,False,False,False,False,False,False
66123,False,False,False,False,False,False,False,False,False,False,False,False,False,False


In [22]:
a = dfIsNan.dtypes

In [23]:
str(a)

'a1_rank_isnan      bool\na2_rank_isnan      bool\na3_rank_isnan      bool\na4_rank_isnan      bool\na4_weapon_isnan    bool\na4_level_isnan     bool\nb1_rank_isnan      bool\nb2_rank_isnan      bool\nb3_rank_isnan      bool\nb4_rank_isnan      bool\nb3_weapon_isnan    bool\nb3_level_isnan     bool\nb4_weapon_isnan    bool\nb4_level_isnan     bool\ndtype: object'

In [24]:
a

a1_rank_isnan      bool
a2_rank_isnan      bool
a3_rank_isnan      bool
a4_rank_isnan      bool
a4_weapon_isnan    bool
a4_level_isnan     bool
b1_rank_isnan      bool
b2_rank_isnan      bool
b3_rank_isnan      bool
b4_rank_isnan      bool
b3_weapon_isnan    bool
b3_level_isnan     bool
b4_weapon_isnan    bool
b4_level_isnan     bool
dtype: object

In [34]:
for clmnNm in dfIsNan:
    print(clmnNm + ": " + str(dfIsNan.dtypes[clmnNm]))

a1_rank_isnan: bool
a2_rank_isnan: bool
a3_rank_isnan: bool
a4_rank_isnan: bool
a4_weapon_isnan: bool
a4_level_isnan: bool
b1_rank_isnan: bool
b2_rank_isnan: bool
b3_rank_isnan: bool
b4_rank_isnan: bool
b3_weapon_isnan: bool
b3_level_isnan: bool
b4_weapon_isnan: bool
b4_level_isnan: bool


In [36]:
for clmnNm in df:
    print(clmnNm + ": " + str(df.dtypes[clmnNm]))

id: int64
lobby_mode: object
mode: object
stage: object
a1_weapon: object
a1_rank: object
a1_level: int64
a2_weapon: object
a2_rank: object
a2_level: int64
a3_weapon: object
a3_rank: object
a3_level: int64
a4_weapon: object
a4_rank: object
a4_level: int64
b1_weapon: object
b1_rank: object
b1_level: int64
b2_weapon: object
b2_rank: object
b2_level: int64
b3_weapon: object
b3_rank: object
b3_level: int64
b4_weapon: object
b4_rank: object
b4_level: int64
yyyymmdd: object
time: object
datedelta: int64
hour: int64
y: int64


In [16]:
dfIsNan.columns

Index(['a1_rank_isnan', 'a2_rank_isnan', 'a3_rank_isnan', 'a4_rank_isnan',
       'a4_weapon_isnan', 'a4_level_isnan', 'b1_rank_isnan', 'b2_rank_isnan',
       'b3_rank_isnan', 'b4_rank_isnan', 'b3_weapon_isnan', 'b3_level_isnan',
       'b4_weapon_isnan', 'b4_level_isnan'],
      dtype='object')

In [17]:
pd.DataFrame(dfIsNan.columns,columns=["clmnNm"])

,clmnNm
0,a1_rank_isnan
1,a2_rank_isnan
2,a3_rank_isnan
3,a4_rank_isnan
4,a4_weapon_isnan
5,a4_level_isnan
6,b1_rank_isnan
7,b2_rank_isnan
8,b3_rank_isnan
9,b4_rank_isnan


In [ ]:
#指定列のUpdate
#Input：DB名、テーブル名、追加するtable(dataframe)、主キーとなるtable(dataframe)
#Output：なし
#以下のルールで追加列の型付けを行う。
#  bool -> smallint: 0 or 1
#  object -> varchar[Max値]
#  intを含む -> int8
#  float -> float8
def addColumns(dbname, tableName, addDf, pkeyDf):
    #queryの枕詞
    iniQrySnpt = "ALTER TABLE " + tableName + " ADD COLUMN "
    
    
    #指定列のadd実行
    with psycopg2.connect("host=localhost port=5432 dbname=" + dbname + " user=tidal password=tidalryoku") as conn:
        with conn.cursor() as cur: